In [2]:
import cv2
import tensorflow as tf
from sklearn import datasets, model_selection, metrics
import trimesh
import open3d as o3d
import numpy as np
from tensorflow import keras
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# Pedir el nombre del objeto
object_name = input("Introduce el nombre del objeto: ")

# Crear la carpeta si no existe
if not os.path.exists(object_name):
    os.makedirs(object_name)

# Iniciar la cámara
cap = cv2.VideoCapture(0)

image_count = len(os.listdir(object_name))  # Contar el número de imágenes existentes en la carpeta

while True:
    # Capturar frame por frame
    ret, frame = cap.read()
    
    # Mostrar el frame resultante
    cv2.imshow('Frame', frame)
    
    # Verificar si se presionó la tecla 'e' para guardar las imágenes
    if cv2.waitKey(1) & 0xFF == ord('e'):
        for i in range(15):  # Tomar 5 fotos
            ret, frame = cap.read()
            cv2.imwrite(f"{object_name}/image_{image_count}.jpg", frame)
            image_count += 1
    
    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la cámara y cerrar todas las ventanas
cap.release()
cv2.destroyAllWindows()


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Función para cargar imágenes desde una carpeta y aplicar la máscara
def load_images_from_folder(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            # Convertir la imagen a escala de grises
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            # Aplicar un umbral para crear una máscara binaria
            _, mask = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)
            
            # Encontrar contornos
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            # Encontrar el contorno más grande
            largest_contour = max(contours, key=cv2.contourArea)
            
            # Crear una máscara con el contorno más grande
            object_mask = np.zeros_like(mask)
            cv2.drawContours(object_mask, [largest_contour], -1, 255, thickness=cv2.FILLED)
            
            # Aplicar la máscara a la imagen original
            object_only = cv2.bitwise_and(img, img, mask=object_mask)
            
            # Añadir la imagen procesada y la etiqueta a las listas
            images.append(object_only)
            labels.append(folder)
    return images, labels

# Pedir el nombre de la carpeta
folder_name = input("Introduce el nombre de la carpeta con las imágenes: ")

# Cargar imágenes y etiquetas
images, labels = load_images_from_folder(folder_name)

# Convertir listas a arrays de numpy
images = np.array(images)
labels = np.array(labels)

# Preprocesar las imágenes
images = images / 255.0  # Normalizar las imágenes

# Codificar las etiquetas
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Crear el modelo
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(images.shape[1], images.shape[2], images.shape[3])),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(lb.classes_), activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Guardar el modelo
model.save('object_recognition_model.h5')


Epoch 1/10
3/3 [==============================] - 20s 5s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
3/3 [==============================] - 14s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
3/3 [==============================] - 14s 5s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
3/3 [==============================] - 14s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
3/3 [==============================] - 13s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
3/3 [==============================] - 13s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
3/3 [==============================] - 13s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 -

In [10]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import os
import time

# Cargar el modelo
model = load_model('object_recognition_model.h5')

# Crear la carpeta de reconocimiento si no existe
if not os.path.exists('reconocimiento'):
    os.makedirs('reconocimiento')

# Iniciar la cámara
cap = cv2.VideoCapture(0)

# Función para preprocesar la imagen
def preprocess_image(image):
    resized_image = cv2.resize(image, (640, 480))  # Redimensionar la imagen a (480, 640)
    preprocessed_image = resized_image / 255.0  # Normalizar la imagen
    preprocessed_image = np.expand_dims(preprocessed_image, axis=0)  # Agregar una dimensión adicional para el batch
    return preprocessed_image

# Función para tomar una ráfaga de 5 fotos
def take_burst():
    for i in range(5):  # Tomar 5 fotos en ráfaga
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocesar la imagen
        preprocessed_image = preprocess_image(frame)

        # Realizar la predicción
        prediction = model.predict(preprocessed_image)
        predicted_class = np.argmax(prediction, axis=1)
        predicted_label = lb.classes_[predicted_class[0]]

        # Convertir la imagen a escala de grises y detectar bordes
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 50, 150)

        # Encontrar contornos
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Si se encontraron contornos
        if contours:
            # Seleccionar el contorno más grande (el objeto principal)
            largest_contour = max(contours, key=cv2.contourArea)

            # Obtener el rectángulo delimitador del contorno más grande
            x, y, w, h = cv2.boundingRect(largest_contour)

            # Recortar la región de interés (ROI) del frame
            roi = frame[y:y+h, x:x+w]

            # Preprocesar la ROI
            preprocessed_roi = preprocess_image(roi)

            # Realizar la predicción en la ROI
            roi_prediction = model.predict(preprocessed_roi)
            roi_predicted_class = np.argmax(roi_prediction, axis=1)
            roi_predicted_label = lb.classes_[roi_predicted_class[0]]

            # Si la etiqueta predicha en la ROI coincide con la etiqueta predicha en el frame completo
            if roi_predicted_label == predicted_label:
                # Crear una máscara para aislar el objeto
                mask = np.zeros_like(gray)  # Crear una máscara en blanco
                cv2.drawContours(mask, [largest_contour], -1, 255, -1)  # Dibujar el contorno en la máscara

                # Aplicar la máscara para aislar el objeto
                object_only = cv2.bitwise_and(frame, frame, mask=mask)

                # Recortar la imagen para guardar solo el objeto
                object_cropped = object_only[y:y+h, x:x+w]

                # Guardar la imagen del objeto recortado
                cv2.imwrite(f"reconocimiento/object_burst_{burst_count}_{i}.jpg", object_cropped)
                print(f"Foto {i + 1} de la ráfaga {burst_count + 1} guardada.")

        # Mostrar el frame en tiempo real (opcional)
        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):  # Salir si se presiona 'q'
            return False

    return True

# Contador de ráfagas
burst_count = 0

# Bucle principal
try:
    while True:
        print(f"Iniciando ráfaga {burst_count + 1}...")
        if not take_burst():  # Tomar una ráfaga de 5 fotos
            break  # Salir si el usuario presiona 'q'

        burst_count += 1
        print(f"Esperando 5 segundos antes de la siguiente ráfaga...")
        time.sleep(5)  # Esperar 5 segundos antes de la siguiente ráfaga

except KeyboardInterrupt:
    print("Proceso interrumpido por el usuario.")

# Liberar la cámara y cerrar todas las ventanas
cap.release()
cv2.destroyAllWindows()

Iniciando ráfaga 1...
1/1 [==============================] - 0s 296ms/step
Foto 1 de la ráfaga 1 guardada.
1/1 [==============================] - 0s 492ms/step
Foto 2 de la ráfaga 1 guardada.
1/1 [==============================] - 0s 240ms/step
Foto 3 de la ráfaga 1 guardada.
1/1 [==============================] - 0s 233ms/step
Foto 4 de la ráfaga 1 guardada.
1/1 [==============================] - 0s 242ms/step
Foto 5 de la ráfaga 1 guardada.
Esperando 5 segundos antes de la siguiente ráfaga...
Iniciando ráfaga 2...
1/1 [==============================] - 0s 213ms/step
Foto 1 de la ráfaga 2 guardada.
1/1 [==============================] - 0s 267ms/step
Foto 2 de la ráfaga 2 guardada.
1/1 [==============================] - 0s 285ms/step
Foto 3 de la ráfaga 2 guardada.
1/1 [==============================] - 0s 253ms/step
Foto 4 de la ráfaga 2 guardada.
1/1 [==============================] - 0s 174ms/step
Foto 5 de la ráfaga 2 guardada.
Esperando 5 segundos antes de la siguiente ráfaga...
